# Make a Good Model for the mood of the song

In this document I need to work more with NLP of the track lyrics and use columns that I discarded in the MVP and I also need to do  some feature engineering.

Better model = countries<BR />
Best model = also looking at number of streams and position, also looking at time on top list<BR />

Then when we have the best model we can use our predictions and decide the mood of a country and the mood of an artist. Then we can say what artist is suitable for what country. 

## Import stuff

In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
import sklearn.metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Load data

In [2]:
data = pd.read_csv('./data_top10c_more_lyrics.csv')

data.head(5)

,Unnamed: 0,Position,Streams,Track Name,Artist,ID,Date,Year,Month,Day,Country,Region,Lyrics,Acousticness,Energy,Instrumentalness,Mode,Tempo,Valence
0,0,177,40381,Bye Bye Bye,*NSYNC,4r8lRYnoOGdEi6YyI5OC1o,2017-10-05,2017,10,5,gb,eu,"hey, hey bye bye bye, bye bye bye bye i'm doi...",0.0408,0.928,0.00104,0.0,172.656,0.879
1,1,151,24132,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-23,2017,12,23,it,eu,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756
2,2,78,49766,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-24,2017,12,24,it,eu,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756
3,3,65,40725,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-25,2017,12,25,it,eu,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756
4,4,197,215739,"Merry Christmas, Happy Holidays",*NSYNC,15coTBAzEN1bOeipoNDZAR,2017-12-22,2017,12,22,us,na,merry christmas and happy holidays merry chris...,0.1030,0.939,0.00000,1.0,105.003,0.756


## Fix a little bit with the data

### Look at each country individually

In [7]:
data['Country'].unique()

array(['gb', 'it', 'us', 'de', 'ca', 'nl', 'au', 'fr'], dtype=object)

In [9]:
# filter by country and make 8 new data frames
gb_data = data[data['Country'] == 'gb']
it_data = data[data['Country'] == 'it']
us_data = data[data['Country'] == 'us']
de_data = data[data['Country'] == 'de']
ca_data = data[data['Country'] == 'ca']
nl_data = data[data['Country'] == 'nl']
au_data = data[data['Country'] == 'au']
fr_data = data[data['Country'] == 'fr']

**Drop rows that are duplicates and keep only one row for each song**

In [10]:
gb_data_per_song = gb_data.drop_duplicates(subset=['Track Name'], keep='first')
it_data_per_song = it_data.drop_duplicates(subset=['Track Name'], keep='first')
us_data_per_song = us_data.drop_duplicates(subset=['Track Name'], keep='first')
de_data_per_song = de_data.drop_duplicates(subset=['Track Name'], keep='first')
ca_data_per_song = ca_data.drop_duplicates(subset=['Track Name'], keep='first')
nl_data_per_song = nl_data.drop_duplicates(subset=['Track Name'], keep='first')
au_data_per_song = au_data.drop_duplicates(subset=['Track Name'], keep='first')
fr_data_per_song = fr_data.drop_duplicates(subset=['Track Name'], keep='first')

**Drop all columns that might change per song**

In [11]:
gb_nlp_data = gb_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
it_nlp_data = it_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
us_nlp_data = us_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
de_nlp_data = de_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
ca_nlp_data = ca_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
nl_nlp_data = nl_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
au_nlp_data = au_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)
fr_nlp_data = fr_data_per_song.drop(['Unnamed: 0', 'Position', 'Streams', 'Date', 'Year', 'Month', 'Day', 'Country',
                                   'Region'], axis=1)

**Drop rows that have missing values in the Lyrics column**<BR />
We can use dropna to drop all rows that has missing values (should mostly be the Lyrics column)

In [13]:
gb_clean = gb_nlp_data.dropna(axis=0, how='any')
it_clean = it_nlp_data.dropna(axis=0, how='any')
us_clean = us_nlp_data.dropna(axis=0, how='any')
de_clean = de_nlp_data.dropna(axis=0, how='any')
ca_clean = ca_nlp_data.dropna(axis=0, how='any')
nl_clean = nl_nlp_data.dropna(axis=0, how='any')
au_clean = au_nlp_data.dropna(axis=0, how='any')
fr_clean = fr_nlp_data.dropna(axis=0, how='any')

### TextBlob

**Turn the lyrics in the Lyrics column into string**

In [14]:
gb_clean['Lyrics'] = gb_clean['Lyrics'].astype(str)
it_clean['Lyrics'] = it_clean['Lyrics'].astype(str)
us_clean['Lyrics'] = us_clean['Lyrics'].astype(str)
de_clean['Lyrics'] = de_clean['Lyrics'].astype(str)
ca_clean['Lyrics'] = ca_clean['Lyrics'].astype(str)
nl_clean['Lyrics'] = nl_clean['Lyrics'].astype(str)
au_clean['Lyrics'] = au_clean['Lyrics'].astype(str)
fr_clean['Lyrics'] = fr_clean['Lyrics'].astype(str)

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

**Make and run function for TextBlob on the Lyrics**

In [15]:
def sentiment_func(lyrics):
    try:
        return TextBlob(lyrics).sentiment
    except:
        return None

In [16]:
gb_clean['pol_sub'] = gb_clean['Lyrics'].apply(sentiment_func)
it_clean['pol_sub'] = it_clean['Lyrics'].apply(sentiment_func)
us_clean['pol_sub'] = us_clean['Lyrics'].apply(sentiment_func)
de_clean['pol_sub'] = de_clean['Lyrics'].apply(sentiment_func)
ca_clean['pol_sub'] = ca_clean['Lyrics'].apply(sentiment_func)
nl_clean['pol_sub'] = nl_clean['Lyrics'].apply(sentiment_func)
au_clean['pol_sub'] = au_clean['Lyrics'].apply(sentiment_func)
fr_clean['pol_sub'] = fr_clean['Lyrics'].apply(sentiment_func)

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

**Split the pol_sub column into 2 new columns (Polarity, Subjectivity)**

In [24]:
gb_clean['Polarity'] = gb_clean['pol_sub'].apply(lambda x: x[0])
gb_clean['Subjectivity'] = gb_clean['pol_sub'].apply(lambda x: x[1])

it_clean['Polarity'] = it_clean['pol_sub'].apply(lambda x: x[0])
it_clean['Subjectivity'] = it_clean['pol_sub'].apply(lambda x: x[1])

us_clean['Polarity'] = us_clean['pol_sub'].apply(lambda x: x[0])
us_clean['Subjectivity'] = us_clean['pol_sub'].apply(lambda x: x[1])

de_clean['Polarity'] = de_clean['pol_sub'].apply(lambda x: x[0])
de_clean['Subjectivity'] = de_clean['pol_sub'].apply(lambda x: x[1])

ca_clean['Polarity'] = ca_clean['pol_sub'].apply(lambda x: x[0])
ca_clean['Subjectivity'] = ca_clean['pol_sub'].apply(lambda x: x[1])

nl_clean['Polarity'] = nl_clean['pol_sub'].apply(lambda x: x[0])
nl_clean['Subjectivity'] = nl_clean['pol_sub'].apply(lambda x: x[1])

au_clean['Polarity'] = au_clean['pol_sub'].apply(lambda x: x[0])
au_clean['Subjectivity'] = au_clean['pol_sub'].apply(lambda x: x[1])

fr_clean['Polarity'] = fr_clean['pol_sub'].apply(lambda x: x[0])
fr_clean['Subjectivity'] = fr_clean['pol_sub'].apply(lambda x: x[1])

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

**Drop the pol_sub column**

In [26]:
gb_clean = gb_clean.drop(['pol_sub'], axis=1)
it_clean = it_clean.drop(['pol_sub'], axis=1)
us_clean = us_clean.drop(['pol_sub'], axis=1)
de_clean = de_clean.drop(['pol_sub'], axis=1)
ca_clean = ca_clean.drop(['pol_sub'], axis=1)
nl_clean = nl_clean.drop(['pol_sub'], axis=1)
au_clean = au_clean.drop(['pol_sub'], axis=1)
fr_clean = fr_clean.drop(['pol_sub'], axis=1)

### Do a quick check of the entire data frame

In [27]:
gb_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041 entries, 0 to 578617
Data columns (total 12 columns):
Track Name          1041 non-null object
Artist              1041 non-null object
ID                  1041 non-null object
Lyrics              1041 non-null object
Acousticness        1041 non-null float64
Energy              1041 non-null float64
Instrumentalness    1041 non-null float64
Mode                1041 non-null float64
Tempo               1041 non-null float64
Valence             1041 non-null float64
Polarity            1041 non-null float64
Subjectivity        1041 non-null float64
dtypes: float64(8), object(4)
memory usage: 145.7+ KB


In [28]:
it_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 952 entries, 1 to 578920
Data columns (total 12 columns):
Track Name          952 non-null object
Artist              952 non-null object
ID                  952 non-null object
Lyrics              952 non-null object
Acousticness        952 non-null float64
Energy              952 non-null float64
Instrumentalness    952 non-null float64
Mode                952 non-null float64
Tempo               952 non-null float64
Valence             952 non-null float64
Polarity            952 non-null float64
Subjectivity        952 non-null float64
dtypes: float64(8), object(4)
memory usage: 136.7+ KB


In [29]:
us_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908 entries, 4 to 578926
Data columns (total 12 columns):
Track Name          908 non-null object
Artist              908 non-null object
ID                  908 non-null object
Lyrics              908 non-null object
Acousticness        908 non-null float64
Energy              908 non-null float64
Instrumentalness    908 non-null float64
Mode                908 non-null float64
Tempo               908 non-null float64
Valence             908 non-null float64
Polarity            908 non-null float64
Subjectivity        908 non-null float64
dtypes: float64(8), object(4)
memory usage: 92.2+ KB


In [30]:
de_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1020 entries, 8 to 577856
Data columns (total 12 columns):
Track Name          1020 non-null object
Artist              1020 non-null object
ID                  1020 non-null object
Lyrics              1020 non-null object
Acousticness        1020 non-null float64
Energy              1020 non-null float64
Instrumentalness    1020 non-null float64
Mode                1020 non-null float64
Tempo               1020 non-null float64
Valence             1020 non-null float64
Polarity            1020 non-null float64
Subjectivity        1020 non-null float64
dtypes: float64(8), object(4)
memory usage: 103.6+ KB


In [31]:
ca_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 801 entries, 32 to 578615
Data columns (total 12 columns):
Track Name          801 non-null object
Artist              801 non-null object
ID                  801 non-null object
Lyrics              801 non-null object
Acousticness        801 non-null float64
Energy              801 non-null float64
Instrumentalness    801 non-null float64
Mode                801 non-null float64
Tempo               801 non-null float64
Valence             801 non-null float64
Polarity            801 non-null float64
Subjectivity        801 non-null float64
dtypes: float64(8), object(4)
memory usage: 81.4+ KB


In [32]:
nl_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 36 to 575613
Data columns (total 12 columns):
Track Name          887 non-null object
Artist              887 non-null object
ID                  887 non-null object
Lyrics              887 non-null object
Acousticness        887 non-null float64
Energy              887 non-null float64
Instrumentalness    887 non-null float64
Mode                887 non-null float64
Tempo               887 non-null float64
Valence             887 non-null float64
Polarity            887 non-null float64
Subjectivity        887 non-null float64
dtypes: float64(8), object(4)
memory usage: 90.1+ KB


In [33]:
au_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 41 to 578619
Data columns (total 12 columns):
Track Name          646 non-null object
Artist              646 non-null object
ID                  646 non-null object
Lyrics              646 non-null object
Acousticness        646 non-null float64
Energy              646 non-null float64
Instrumentalness    646 non-null float64
Mode                646 non-null float64
Tempo               646 non-null float64
Valence             646 non-null float64
Polarity            646 non-null float64
Subjectivity        646 non-null float64
dtypes: float64(8), object(4)
memory usage: 65.6+ KB


In [34]:
fr_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1238 entries, 48 to 578929
Data columns (total 12 columns):
Track Name          1238 non-null object
Artist              1238 non-null object
ID                  1238 non-null object
Lyrics              1238 non-null object
Acousticness        1238 non-null float64
Energy              1238 non-null float64
Instrumentalness    1238 non-null float64
Mode                1238 non-null float64
Tempo               1238 non-null float64
Valence             1238 non-null float64
Polarity            1238 non-null float64
Subjectivity        1238 non-null float64
dtypes: float64(8), object(4)
memory usage: 125.7+ KB


## Train/Test-split

Divide the data into a train and a test set (with a test set of 25%, which is also default)

In [35]:
gb_dep   = gb_clean['Valence']
gb_indep = gb_clean
gb_indep_train, gb_indep_test, gb_dep_train, gb_dep_test = train_test_split(gb_indep, gb_dep, test_size = 0.25,
                                                                            random_state=24)

it_dep   = it_clean['Valence']
it_indep = it_clean
it_indep_train, it_indep_test, it_dep_train, it_dep_test = train_test_split(it_indep, it_dep, test_size = 0.25,
                                                                            random_state=24)

us_dep   = us_clean['Valence']
us_indep = us_clean
us_indep_train, us_indep_test, us_dep_train, us_dep_test = train_test_split(us_indep, us_dep, test_size = 0.25,
                                                                            random_state=24)

de_dep   = de_clean['Valence']
de_indep = de_clean
de_indep_train, de_indep_test, de_dep_train, de_dep_test = train_test_split(de_indep, de_dep, test_size = 0.25,
                                                                            random_state=24)

ca_dep   = ca_clean['Valence']
ca_indep = ca_clean
ca_indep_train, ca_indep_test, ca_dep_train, ca_dep_test = train_test_split(ca_indep, ca_dep, test_size = 0.25,
                                                                            random_state=24)

nl_dep   = nl_clean['Valence']
nl_indep = nl_clean
nl_indep_train, nl_indep_test, nl_dep_train, nl_dep_test = train_test_split(nl_indep, nl_dep, test_size = 0.25,
                                                                            random_state=24)

au_dep   = au_clean['Valence']
au_indep = au_clean
au_indep_train, au_indep_test, au_dep_train, au_dep_test = train_test_split(au_indep, au_dep, test_size = 0.25,
                                                                            random_state=24)

fr_dep   = fr_clean['Valence']
fr_indep = fr_clean
fr_indep_train, fr_indep_test, fr_dep_train, fr_dep_test = train_test_split(fr_indep, fr_dep, test_size = 0.25,
                                                                            random_state=24)

## NLP

In [44]:
# read in csv with stop words
stop_words_df = pd.read_csv('./stop_words.csv')

In [45]:
# turn the df to list
stop_words = stop_words_df['colummn'].tolist()

### CountVectorizer (use in model 1)

In [58]:
# instantiate the model
cvec = CountVectorizer(stop_words = stop_words, max_features = 1000) 

GB

In [59]:
# fit the count vectorizer with training data. 
cvec.fit(gb_indep_train['Lyrics'])
# transform X_train
gb_cvec = cvec.transform(gb_indep_train['Lyrics'])
# Turn the features into a data frame
gb_df  = pd.DataFrame(gb_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
gb_indep_train_cvec = pd.concat([gb_indep_train.reset_index(drop=True), gb_df], axis=1)
# transform X_test
gb_cvec2 = cvec.transform(gb_indep_test['Lyrics'])
# Turn the features into a data frame
gb_df2  = pd.DataFrame(gb_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
gb_indep_test_cvec = pd.concat([gb_indep_test.reset_index(drop=True), gb_df2 ], axis=1)

IT

In [60]:
# fit the count vectorizer with training data. 
cvec.fit(it_indep_train['Lyrics'])
# transform X_train
it_cvec = cvec.transform(it_indep_train['Lyrics'])
# Turn the features into a data frame
it_df  = pd.DataFrame(it_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
it_indep_train_cvec = pd.concat([it_indep_train.reset_index(drop=True), it_df], axis=1)
# transform X_test
it_cvec2 = cvec.transform(it_indep_test['Lyrics'])
# Turn the features into a data frame
it_df2  = pd.DataFrame(it_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
it_indep_test_cvec = pd.concat([it_indep_test.reset_index(drop=True), it_df2 ], axis=1)

US

In [61]:
# fit the count vectorizer with training data. 
cvec.fit(us_indep_train['Lyrics'])
# transform X_train
us_cvec = cvec.transform(us_indep_train['Lyrics'])
# Turn the features into a data frame
us_df  = pd.DataFrame(us_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
us_indep_train_cvec = pd.concat([us_indep_train.reset_index(drop=True), us_df], axis=1)
# transform X_test
us_cvec2 = cvec.transform(us_indep_test['Lyrics'])
# Turn the features into a data frame
us_df2  = pd.DataFrame(us_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
us_indep_test_cvec = pd.concat([us_indep_test.reset_index(drop=True), us_df2 ], axis=1)

DE

In [62]:
# fit the count vectorizer with training data. 
cvec.fit(de_indep_train['Lyrics'])
# transform X_train
de_cvec = cvec.transform(de_indep_train['Lyrics'])
# Turn the features into a data frame
de_df  = pd.DataFrame(de_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
de_indep_train_cvec = pd.concat([de_indep_train.reset_index(drop=True), de_df], axis=1)
# transform X_test
de_cvec2 = cvec.transform(de_indep_test['Lyrics'])
# Turn the features into a data frame
de_df2  = pd.DataFrame(de_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
de_indep_test_cvec = pd.concat([de_indep_test.reset_index(drop=True), de_df2 ], axis=1)

CA

In [63]:
# fit the count vectorizer with training data. 
cvec.fit(ca_indep_train['Lyrics'])
# transform X_train
ca_cvec = cvec.transform(ca_indep_train['Lyrics'])
# Turn the features into a data frame
ca_df  = pd.DataFrame(ca_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
ca_indep_train_cvec = pd.concat([ca_indep_train.reset_index(drop=True), ca_df], axis=1)
# transform X_test
ca_cvec2 = cvec.transform(ca_indep_test['Lyrics'])
# Turn the features into a data frame
ca_df2  = pd.DataFrame(ca_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
ca_indep_test_cvec = pd.concat([ca_indep_test.reset_index(drop=True), ca_df2 ], axis=1)

NL

In [64]:
# fit the count vectorizer with training data. 
cvec.fit(nl_indep_train['Lyrics'])
# transform X_train
nl_cvec = cvec.transform(nl_indep_train['Lyrics'])
# Turn the features into a data frame
nl_df  = pd.DataFrame(nl_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
nl_indep_train_cvec = pd.concat([nl_indep_train.reset_index(drop=True), nl_df], axis=1)
# transform X_test
nl_cvec2 = cvec.transform(nl_indep_test['Lyrics'])
# Turn the features into a data frame
nl_df2  = pd.DataFrame(nl_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
nl_indep_test_cvec = pd.concat([nl_indep_test.reset_index(drop=True), nl_df2 ], axis=1)

AU

In [65]:
# fit the count vectorizer with training data. 
cvec.fit(au_indep_train['Lyrics'])
# transform X_train
au_cvec = cvec.transform(au_indep_train['Lyrics'])
# Turn the features into a data frame
au_df  = pd.DataFrame(au_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
au_indep_train_cvec = pd.concat([au_indep_train.reset_index(drop=True), au_df], axis=1)
# transform X_test
au_cvec2 = cvec.transform(au_indep_test['Lyrics'])
# Turn the features into a data frame
au_df2  = pd.DataFrame(au_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
au_indep_test_cvec = pd.concat([au_indep_test.reset_index(drop=True), au_df2 ], axis=1)

FR

In [66]:
# fit the count vectorizer with training data. 
cvec.fit(fr_indep_train['Lyrics'])
# transform X_train
fr_cvec = cvec.transform(fr_indep_train['Lyrics'])
# Turn the features into a data frame
fr_df  = pd.DataFrame(fr_cvec.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for fitting the model
fr_indep_train_cvec = pd.concat([fr_indep_train.reset_index(drop=True), fr_df], axis=1)
# transform X_test
fr_cvec2 = cvec.transform(fr_indep_test['Lyrics'])
# Turn the features into a data frame
fr_df2  = pd.DataFrame(fr_cvec2.todense(),columns=cvec.get_feature_names())
# Concat with big data frame and use for scoring the model
fr_indep_test_cvec = pd.concat([fr_indep_test.reset_index(drop=True), fr_df2 ], axis=1)

### TF-IDF (use in model 2)

In [67]:
# instantiate the model
tvec = TfidfVectorizer(stop_words = stop_words, max_features = 1000) 

GB

In [68]:
# fit the count vectorizer with training data. 
tvec.fit(gb_indep_train['Lyrics'])
# transform X_train
gb_tvec = tvec.transform(gb_indep_train['Lyrics'])
# Turn the features into a data frame
gb_df  = pd.DataFrame(gb_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
gb_indep_train_tvec = pd.concat([gb_indep_train.reset_index(drop=True), gb_df], axis=1)
# transform X_test
gb_tvec2 = tvec.transform(gb_indep_test['Lyrics'])
# Turn the features into a data frame
gb_df2  = pd.DataFrame(gb_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
gb_indep_test_tvec = pd.concat([gb_indep_test.reset_index(drop=True), gb_df2 ], axis=1)

IT

In [69]:
# fit the count vectorizer with training data. 
tvec.fit(it_indep_train['Lyrics'])
# transform X_train
it_tvec = tvec.transform(it_indep_train['Lyrics'])
# Turn the features into a data frame
it_df  = pd.DataFrame(it_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
it_indep_train_tvec = pd.concat([it_indep_train.reset_index(drop=True), it_df], axis=1)
# transform X_test
it_tvec2 = tvec.transform(it_indep_test['Lyrics'])
# Turn the features into a data frame
it_df2  = pd.DataFrame(it_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
it_indep_test_tvec = pd.concat([it_indep_test.reset_index(drop=True), it_df2 ], axis=1)

US

In [70]:
# fit the count vectorizer with training data. 
tvec.fit(us_indep_train['Lyrics'])
# transform X_train
us_tvec = tvec.transform(us_indep_train['Lyrics'])
# Turn the features into a data frame
us_df  = pd.DataFrame(us_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
us_indep_train_tvec = pd.concat([us_indep_train.reset_index(drop=True), us_df], axis=1)
# transform X_test
us_tvec2 = tvec.transform(us_indep_test['Lyrics'])
# Turn the features into a data frame
us_df2  = pd.DataFrame(us_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
us_indep_test_tvec = pd.concat([us_indep_test.reset_index(drop=True), us_df2 ], axis=1)

DE

In [71]:
# fit the count vectorizer with training data. 
tvec.fit(de_indep_train['Lyrics'])
# transform X_train
de_tvec = tvec.transform(de_indep_train['Lyrics'])
# Turn the features into a data frame
de_df  = pd.DataFrame(de_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
de_indep_train_tvec = pd.concat([de_indep_train.reset_index(drop=True), de_df], axis=1)
# transform X_test
de_tvec2 = tvec.transform(de_indep_test['Lyrics'])
# Turn the features into a data frame
de_df2  = pd.DataFrame(de_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
de_indep_test_tvec = pd.concat([de_indep_test.reset_index(drop=True), de_df2 ], axis=1)

CA

In [72]:
# fit the count vectorizer with training data. 
tvec.fit(ca_indep_train['Lyrics'])
# transform X_train
ca_tvec = tvec.transform(ca_indep_train['Lyrics'])
# Turn the features into a data frame
ca_df  = pd.DataFrame(ca_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
ca_indep_train_tvec = pd.concat([ca_indep_train.reset_index(drop=True), ca_df], axis=1)
# transform X_test
ca_tvec2 = tvec.transform(ca_indep_test['Lyrics'])
# Turn the features into a data frame
ca_df2  = pd.DataFrame(ca_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
ca_indep_test_tvec = pd.concat([ca_indep_test.reset_index(drop=True), ca_df2 ], axis=1)

NL

In [73]:
# fit the count vectorizer with training data. 
tvec.fit(nl_indep_train['Lyrics'])
# transform X_train
nl_tvec = tvec.transform(nl_indep_train['Lyrics'])
# Turn the features into a data frame
nl_df  = pd.DataFrame(nl_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
nl_indep_train_tvec = pd.concat([nl_indep_train.reset_index(drop=True), nl_df], axis=1)
# transform X_test
nl_tvec2 = tvec.transform(nl_indep_test['Lyrics'])
# Turn the features into a data frame
nl_df2  = pd.DataFrame(nl_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
nl_indep_test_tvec = pd.concat([nl_indep_test.reset_index(drop=True), nl_df2 ], axis=1)

AU

In [74]:
# fit the count vectorizer with training data. 
tvec.fit(au_indep_train['Lyrics'])
# transform X_train
au_tvec = tvec.transform(au_indep_train['Lyrics'])
# Turn the features into a data frame
au_df  = pd.DataFrame(au_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
au_indep_train_tvec = pd.concat([au_indep_train.reset_index(drop=True), au_df], axis=1)
# transform X_test
au_tvec2 = tvec.transform(au_indep_test['Lyrics'])
# Turn the features into a data frame
au_df2  = pd.DataFrame(au_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
au_indep_test_tvec = pd.concat([au_indep_test.reset_index(drop=True), au_df2 ], axis=1)

FR

In [75]:
# fit the count vectorizer with training data. 
tvec.fit(fr_indep_train['Lyrics'])
# transform X_train
fr_tvec = tvec.transform(fr_indep_train['Lyrics'])
# Turn the features into a data frame
fr_df  = pd.DataFrame(fr_tvec.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for fitting the model
fr_indep_train_tvec = pd.concat([fr_indep_train.reset_index(drop=True), fr_df], axis=1)
# transform X_test
fr_tvec2 = tvec.transform(fr_indep_test['Lyrics'])
# Turn the features into a data frame
fr_df2  = pd.DataFrame(fr_tvec2.todense(),columns=tvec.get_feature_names())
# Concat with big data frame and use for scoring the model
fr_indep_test_tvec = pd.concat([fr_indep_test.reset_index(drop=True), fr_df2 ], axis=1)

## Models (Lasso) - CountVec

I did choose Lasso bc that gave the best score when looking at all countries togehter.<BR />
If the Lasso scores very well in one country (compared to the total score) then I might consider trying other models.

**Lasso function (best hyperparameters to use and evauation of the model)**

In [78]:
def get_best_hype(model, params, X_train, y_train):  
    # standardize
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_s = ss.transform(X_train)
    
    # Best Hyperparameters
    gs = GridSearchCV(model, grid) # cv default = 3
    
    # fit
    gs.fit(X_train_s, y_train)
     
    return {'best_score': gs.best_score_,'best_params': gs.best_params_} 

def evaluate_model(model, X_train, X_test, y_train, y_test):
    # standardize the predictors
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_s = ss.transform(X_train)
    X_test_s = ss.transform(X_test)
    
    # fit
    model.fit(X_train_s, y_train)
    
    # Evaluate: predict
    
    y_pred = model.predict(X_test_s)
    y_true = y_test
    
    mean_square_error = np.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    
    # Evaluate: score
    score = model.score(X_test_s, y_test)
    
    return {'Score (R^2)': score.mean(), 'MSE': mean_square_error}

### Great Brittan / United Kingdom

**Lasso #1 - CountVec + all coefs**

In [84]:
# Declare indep and dep
X_train_gb = gb_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_gb = gb_dep_train
X_test_gb = gb_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_gb = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_gb, y_train_gb)

{'best_params': {'alpha': 0.014174741629268055, 'selection': 'cyclic'},
 'best_score': 0.1683966517723151}

In [85]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0142, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_gb, X_test_gb, y_train_gb, y_test_gb)

{'MSE': 0.21918612906036872, 'Score (R^2)': 0.12372100674644926}

In [88]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_gb.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy    0.072306
lies      0.011188
yes       0.010698
nigga     0.009808
hot       0.009049
wanted    0.007884
little    0.007816
kind      0.007759
Mode      0.007620
scared    0.007187
sexy      0.006676
honey     0.006455
ring      0.006309
niggas    0.005751
lovers    0.005324
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [89]:
# Declare indep and dep
X_train_gb2 = gb_indep_train_cvec[['Energy', 'lies', 'yes', 'nigga', 'hot', 'wanted', 'little', 'kind',
                                'Mode', 'scared']]
y_train_gb2 = gb_dep_train
X_test_gb2 = gb_indep_test_cvec[['Energy', 'lies', 'yes', 'nigga', 'hot', 'wanted', 'little', 'kind',
                                'Mode', 'scared']]
y_test_gb2 = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_gb2, y_train_gb2)

{'best_params': {'alpha': 0.0013219411484660286, 'selection': 'cyclic'},
 'best_score': 0.21745378864477283}

In [90]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0013, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg2, X_train_gb2, X_test_gb2, y_train_gb2, y_test_gb2)

{'MSE': 0.2797014595389332, 'Score (R^2)': -0.4269400652275712}

**Lasso #3 - CountVec + top 3 coefs**

In [91]:
# Declare indep and dep
X_train_gb3 = gb_indep_train_cvec[['Energy', 'lies', 'yes']]
y_train_gb3 = gb_dep_train
X_test_gb3 = gb_indep_test_cvec[['Energy', 'lies', 'yes']]
y_test_gb3 = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_gb3, y_train_gb3)

{'best_params': {'alpha': 0.0013219411484660286, 'selection': 'random'},
 'best_score': 0.16102250768577955}

In [92]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0013, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_gb3, X_test_gb3, y_train_gb3, y_test_gb3)

{'MSE': 0.21418328472846523, 'Score (R^2)': 0.16326600066483032}

COMMENT: Best Lasso model for GB is nr 3

### Italy

**Lasso #1 - CountVec + all coefs**

In [93]:
# Declare indep and dep
X_train_it = it_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_it = it_dep_train
X_test_it = it_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_it = it_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_it, y_train_it)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'cyclic'},
 'best_score': 0.10361454849115626}

In [94]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0163, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_it, X_test_it, y_train_it, y_test_it)

{'MSE': 0.19770016544111002, 'Score (R^2)': 0.129795914234341}

In [95]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_it.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy       0.054747
oh           0.012264
pa           0.010187
slow         0.008017
ogni         0.007033
yes          0.006925
sing         0.005469
heart        0.005298
mistletoe    0.004306
girl         0.004016
domani       0.003997
colpa        0.003705
pretty       0.003625
tú           0.003604
davvero      0.003454
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [96]:
# Declare indep and dep
X_train_it2 = it_indep_train_cvec[['Energy', 'oh', 'pa', 'slow', 'ogni', 'yes', 'sing', 'heart', 'mistletoe', 'girl']]
y_train_it2 = it_dep_train
X_test_it2 = it_indep_test_cvec[['Energy', 'oh', 'pa', 'slow', 'ogni', 'yes', 'sing', 'heart', 'mistletoe', 'girl']]
y_test_it2 = it_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_it2, y_train_it2)

{'best_params': {'alpha': 0.001747528400007683, 'selection': 'random'},
 'best_score': 0.19150410411859672}

In [99]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0017, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_it2, X_test_it2, y_train_it2, y_test_it2)

{'MSE': 0.1983103291352571, 'Score (R^2)': 0.12441618880693563}

**Lasso #3 - CountVec + top 3 coefs**

In [100]:
# Declare indep and dep
X_train_it3 = it_indep_train_cvec[['Energy', 'oh', 'pa']]
y_train_it3 = it_dep_train
X_test_it3 = it_indep_test_cvec[['Energy', 'oh', 'pa']]
y_test_it3 = it_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_it3, y_train_it3)

{'best_params': {'alpha': 0.001747528400007683, 'selection': 'cyclic'},
 'best_score': 0.12924155327088646}

In [101]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0017, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg3, X_train_it3, X_test_it3, y_train_it3, y_test_it3)

{'MSE': 0.20250334197393632, 'Score (R^2)': 0.0869986023385424}

COMMENT: Best Lasso model for IT is nr 1

### US

**Lasso #1 - CountVec + all coefs**

In [103]:
# Declare indep and dep
X_train_us = us_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_us = us_dep_train
X_test_us = us_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_us = us_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000)

get_best_hype(lassoreg, grid, X_train_us, y_train_us)

{'best_params': {'alpha': 0.012328467394420659, 'selection': 'cyclic'},
 'best_score': 0.16815884107343257}

In [104]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000, alpha=0.0123, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_us, X_test_us, y_train_us, y_test_us)

{'MSE': 0.207156835957949, 'Score (R^2)': 0.1626550940687036}

In [105]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_us.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy     0.076635
baby       0.020060
rollie     0.012212
cake       0.012170
bitch      0.011404
bells      0.010804
ground     0.010176
askin      0.008671
sure       0.008383
sing       0.007925
sheets     0.007804
worry      0.007282
nick       0.006946
bands      0.006844
calling    0.006620
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [106]:
# Declare indep and dep
X_train_us2 = us_indep_train_cvec[['Energy', 'baby', 'rollie', 'cake', 'bitch', 'bells', 'ground', 'askin',
                                'sure', 'sing']]
y_train_us2 = us_dep_train
X_test_us2 = us_indep_test_cvec[['Energy', 'baby', 'rollie', 'cake', 'bitch', 'bells', 'ground', 'askin',
                                'sure', 'sing']]
y_test_us2 = us_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_us2, y_train_us2)

{'best_params': {'alpha': 0.005336699231206312, 'selection': 'random'},
 'best_score': 0.23703559962807996}

In [107]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0053, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_us2, X_test_us2, y_train_us2, y_test_us2)

{'MSE': 0.20746232941947448, 'Score (R^2)': 0.16018361386091773}

**Lasso #3 - CountVec + top 3 coefs**

In [108]:
# Declare indep and dep
X_train_us3 = us_indep_train_cvec[['Energy', 'baby', 'rollie']]
y_train_us3 = us_dep_train
X_test_us3 = us_indep_test_cvec[['Energy', 'baby', 'rollie']]
y_test_us3 = us_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_us3, y_train_us3)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.1869515895970676}

In [109]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_us3, X_test_us3, y_train_us3, y_test_us3)

{'MSE': 0.20800162998603408, 'Score (R^2)': 0.15581171530153481}

COMMENT: Best Lasso model for US is nr 1

### Deutchland / Germany

**Lasso #1 - CountVec + all coefs**

In [111]:
# Declare indep and dep
X_train_de = de_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_de = de_dep_train
X_test_de = de_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_de = de_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_de, y_train_de)

{'best_params': {'alpha': 0.014174741629268055, 'selection': 'cyclic'},
 'best_score': 0.08500697940732012}

In [112]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0142, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_de, X_test_de, y_train_de, y_test_de)

{'MSE': 0.19605402659952886, 'Score (R^2)': 0.14787882213504022}

In [113]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_de.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy    0.053430
snow      0.012721
ahh       0.009732
bit       0.008868
moment    0.005708
erst      0.005485
baby      0.005459
viel      0.004376
Mode      0.003797
plan      0.003254
fast      0.003224
uh        0.003008
dicka     0.002914
seele     0.002586
fresh     0.002390
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [115]:
# Declare indep and dep
X_train_de2 = de_indep_train_cvec[['Energy', 'snow', 'ahh', 'bit', 'moment', 'erst', 'baby', 'viel',
                                'Mode', 'plan']]
y_train_de2 = de_dep_train
X_test_de2 = de_indep_test_cvec[['Energy', 'snow', 'ahh', 'bit', 'moment', 'erst', 'baby', 'viel',
                                'Mode', 'plan']]
y_test_de2 = de_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_de2, y_train_de2)

{'best_params': {'alpha': 0.0013219411484660286, 'selection': 'random'},
 'best_score': 0.17855719581864732}

In [116]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0013, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_de2, X_test_de2, y_train_de2, y_test_de2)

{'MSE': 0.19376925752158428, 'Score (R^2)': 0.1676239483701173}

**Lasso #3 - CountVec + top 3 coefs**

In [117]:
# Declare indep and dep
X_train_de3 = de_indep_train_cvec[['Energy', 'snow', 'ahh']]
y_train_de3 = de_dep_train
X_test_de3 = de_indep_test_cvec[['Energy', 'snow', 'ahh']]
y_test_de3 = de_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_de3, y_train_de3)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.12061143715799284}

In [118]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_de3, X_test_de3, y_train_de3, y_test_de3)

{'MSE': 0.19647908136332912, 'Score (R^2)': 0.1441799356190453}

COMMENT: Best Lasso model for DE is nr 2

### Canada

**Lasso #1 - CountVec + all coefs**

In [119]:
# Declare indep and dep
X_train_ca = ca_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_ca = ca_dep_train
X_test_ca = ca_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_ca = ca_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_ca, y_train_ca)

{'best_params': {'alpha': 0.014174741629268055, 'selection': 'cyclic'},
 'best_score': 0.15930869073648657}

In [120]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0142, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_ca, X_test_ca, y_train_ca, y_test_ca)

{'MSE': 0.22657917601179678, 'Score (R^2)': 0.07276780488464796}

In [121]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_ca.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy              0.067558
sing                0.025703
baby                0.013972
Acousticness        0.013553
oh                  0.010626
rollie              0.010614
fucked              0.008410
Instrumentalness    0.007637
year                0.007305
remember            0.006855
brand               0.006680
slow                0.006604
bit                 0.006290
nick                0.005191
snow                0.005035
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [122]:
# Declare indep and dep
X_train_ca2 = ca_indep_train_cvec[['Energy', 'sing', 'baby', 'Acousticness', 'oh', 'rollie', 'fucked', 
                                   'Instrumentalness', 'year', 'remember']]
y_train_ca2 = ca_dep_train
X_test_ca2 = ca_indep_test_cvec[['Energy', 'sing', 'baby', 'Acousticness', 'oh', 'rollie', 'fucked', 
                                'Instrumentalness', 'year', 'remember']]
y_test_ca2 = ca_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_ca2, y_train_ca2)

{'best_params': {'alpha': 0.0023101297000831605, 'selection': 'cyclic'},
 'best_score': 0.26790626157168884}

In [127]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0023, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg2, X_train_ca2, X_test_ca2, y_train_ca2, y_test_ca2)

{'MSE': 0.23197827784145408, 'Score (R^2)': 0.02805171789476601}

**Lasso #3 - CountVec + top 3 coefs**

In [128]:
# Declare indep and dep
X_train_ca3 = ca_indep_train_cvec[['Energy', 'sing', 'baby']]
y_train_ca3 = ca_dep_train
X_test_ca3 = ca_indep_test_cvec[['Energy', 'sing', 'baby']]
y_test_ca3 = ca_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_ca3, y_train_ca3)

{'best_params': {'alpha': 0.0013219411484660286, 'selection': 'random'},
 'best_score': 0.1833856494586246}

In [129]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0013, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_ca3, X_test_ca3, y_train_ca3, y_test_ca3)

{'MSE': 0.2323572971552353, 'Score (R^2)': 0.024873074443244828}

COMMENT: Best Lasso model for CA is nr 1

### Netherlands

**Lasso #1 - CountVec + all coefs**

In [147]:
# Declare indep and dep
X_train_nl = nl_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_nl = nl_dep_train
X_test_nl = nl_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_nl = nl_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000)

get_best_hype(lassoreg, grid, X_train_nl, y_train_nl)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'cyclic'},
 'best_score': 0.1847801802854809}

In [148]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000, alpha=0.0163, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_nl, X_test_nl, y_train_nl, y_test_nl)

{'MSE': 0.20570261363696934, 'Score (R^2)': 0.08232260169784689}

In [149]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_nl.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy     0.078225
Mode       0.011507
bells      0.011294
let        0.008211
saaf       0.006024
weg        0.005755
baby       0.005137
kan        0.004927
life       0.004434
apart      0.003696
praten     0.003185
light      0.003032
sing       0.003026
moest      0.002991
feeling    0.002460
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [150]:
# Declare indep and dep
X_train_nl2 = nl_indep_train_cvec[['Energy', 'Mode', 'bells', 'let', 'saaf', 'weg', 'baby', 'kan',
                                'life', 'apart']]
y_train_nl2 = nl_dep_train
X_test_nl2 = nl_indep_test_cvec[['Energy', 'Mode', 'bells', 'let', 'saaf', 'weg', 'baby', 'kan',
                                'life', 'apart']]
y_test_nl2 = nl_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_nl2, y_train_nl2)

{'best_params': {'alpha': 0.0026560877829466868, 'selection': 'random'},
 'best_score': 0.2446037402505679}

In [151]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0027, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_nl2, X_test_nl2, y_train_nl2, y_test_nl2)

{'MSE': 0.22999339070496982, 'Score (R^2)': -0.1472052549351377}

**Lasso #3 - CountVec + top 3 coefs**

In [152]:
# Declare indep and dep
X_train_nl3 = nl_indep_train_cvec[['Energy', 'Mode', 'bells']]
y_train_nl3 = nl_dep_train
X_test_nl3 = nl_indep_test_cvec[['Energy', 'Mode', 'bells']]
y_test_nl3 = nl_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_nl3, y_train_nl3)

{'best_params': {'alpha': 0.0030538555088334154, 'selection': 'random'},
 'best_score': 0.21151890852905245}

In [153]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0031, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_nl3, X_test_nl3, y_train_nl3, y_test_nl3)

{'MSE': 0.205671658747888, 'Score (R^2)': 0.08259877188557052}

COMMENT: Best Lasso model for NL is nr 3

### Australia

**Lasso #1 - CountVec + all coefs**

In [144]:
# Declare indep and dep
X_train_au = au_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_au = au_dep_train
X_test_au = au_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_au = au_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000)

get_best_hype(lassoreg, grid, X_train_au, y_train_au)

{'best_params': {'alpha': 0.021544346900318846, 'selection': 'cyclic'},
 'best_score': 0.15196701824089573}

In [154]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000, alpha=0.0215, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_au, X_test_au, y_train_au, y_test_au)

{'MSE': 0.2032233116982194, 'Score (R^2)': 0.19147938948164733}

In [155]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_au.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy       0.063603
sing         0.016007
baby         0.008907
sweat        0.006581
ring         0.005776
hey          0.005512
year         0.005325
road         0.004765
attention    0.004345
jump         0.003947
alright      0.003563
ride         0.003460
sip          0.003339
mess         0.003003
says         0.001592
dtype: float64

**Lasso #2 - CountVec + top 10 coefs**

In [156]:
# Declare indep and dep
X_train_au2 = au_indep_train_cvec[['Energy', 'sing', 'baby', 'sweat', 'ring', 'hey', 'year', 'road',
                                'attention', 'jump']]
y_train_au2 = au_dep_train
X_test_au2 = au_indep_test_cvec[['Energy', 'sing', 'baby', 'sweat', 'ring', 'hey', 'year', 'road',
                                'attention', 'jump']]
y_test_au2 = au_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_au2, y_train_au2)

{'best_params': {'alpha': 0.005336699231206312, 'selection': 'random'},
 'best_score': 0.23643307953613585}

In [157]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0053, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_au2, X_test_au2, y_train_au2, y_test_au2)

{'MSE': 0.20500858588565368, 'Score (R^2)': 0.17721162577770821}

**Lasso #3 - CountVec + top 3 coefs**

In [158]:
# Declare indep and dep
X_train_au3 = au_indep_train_cvec[['Energy', 'sing', 'baby']]
y_train_au3 = au_dep_train
X_test_au3 = au_indep_test_cvec[['Energy', 'sing', 'baby']]
y_test_au3 = au_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_au3, y_train_au3)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.1880024050952279}

In [159]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_au3, X_test_au3, y_train_au3, y_test_au3)

{'MSE': 0.20311970915467437, 'Score (R^2)': 0.19230354139175532}

COMMENT: Best Lasso model for AU is nr 3

### France

**Lasso #1 - CountVec + all coefs**

In [160]:
# Declare indep and dep
X_train_fr = fr_indep_train_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_fr = fr_dep_train
X_test_fr = fr_indep_test_cvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_fr = fr_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_fr, y_train_fr)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'cyclic'},
 'best_score': 0.16762695430259927}

In [161]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0163, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_fr, X_test_fr, y_train_fr, y_test_fr)

{'MSE': 0.1965943250668124, 'Score (R^2)': 0.17412071001770535}

In [162]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_fr.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy          0.072582
x2              0.017689
snow            0.009138
fine            0.007774
bout            0.006733
sing            0.005494
quartier        0.005459
Acousticness    0.005119
dancing         0.004101
baby            0.004017
sommet          0.002431
pute            0.002318
rend            0.002135
jul             0.001950
lumière         0.001632
dtype: float64

In [ ]:
# Do not use x2, that is from the lyrics text and only showing if something is repeating

**Lasso #2 - CountVec + top 10 coefs**

In [163]:
# Declare indep and dep
X_train_fr2 = fr_indep_train_cvec[['Energy', 'snow', 'fine', 'bout', 'sing', 'quartier', 'Acousticness', 'dancing',
                                'baby', 'sommet']]
y_train_fr2 = fr_dep_train
X_test_fr2 = fr_indep_test_cvec[['Energy', 'snow', 'fine', 'bout', 'sing', 'quartier', 'Acousticness', 'dancing',
                                'baby', 'sommet']]
y_test_fr2 = fr_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_fr2, y_train_fr2)

{'best_params': {'alpha': 0.0026560877829466868, 'selection': 'cyclic'},
 'best_score': 0.20654440096710708}

In [164]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0027, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg2, X_train_fr2, X_test_fr2, y_train_fr2, y_test_fr2)

{'MSE': 0.20890472878595792, 'Score (R^2)': 0.0674520689541851}

**Lasso #3 - CountVec + top 3 coefs**

In [165]:
# Declare indep and dep
X_train_fr3 = fr_indep_train_cvec[['Energy', 'snow', 'fine']]
y_train_fr3 = fr_dep_train
X_test_fr3 = fr_indep_test_cvec[['Energy', 'snow', 'fine']]
y_test_fr3 = fr_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_fr3, y_train_fr3)

{'best_params': {'alpha': 0.001, 'selection': 'cyclic'},
 'best_score': 0.17040468269503137}

In [166]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0013, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_fr3, X_test_fr3, y_train_fr3, y_test_fr3)

{'MSE': 0.19728395940514648, 'Score (R^2)': 0.16831633403080726}

COMMENT: Best Lasso model for FR is nr 1

**Summary CountVec - best model/score per country**<BR />
gb - R2 = 0.16326600066483032 (top 3 coefs)<BR />
it - R2 = 0.129795914234341 (all coefs)<BR />
us - R2 = 0.1626550940687036 (all coefs)<BR />
de - R2 = 0.1676239483701173 (top 10 coefs)<BR />
ca - R2 = 0.07276780488464796 (all coefs)<BR />
nl - R2 = 0.08259877188557052 (top 3 coefs)<BR />
au - R2 = 0.19230354139175532 (top 3 coefs)<BR />
fr - R2 = 0.17412071001770535 (all coefs)

*The score and how many varables to choose (and which ones) varies from country to country*

## Models (Lasso) - TF-IDF

I did choose Lasso bc that gave the best score when looking at all countries togehter.<BR />
If the Lasso scores very well in one country (compared to the total score) then I might consider trying other models.

**Lasso function (best hyperparameters to use and evauation of the model)**

In [78]:
def get_best_hype(model, params, X_train, y_train):  
    # standardize
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_s = ss.transform(X_train)
    
    # Best Hyperparameters
    gs = GridSearchCV(model, grid) # cv default = 3
    
    # fit
    gs.fit(X_train_s, y_train)
     
    return {'best_score': gs.best_score_,'best_params': gs.best_params_} 

def evaluate_model(model, X_train, X_test, y_train, y_test):
    # standardize the predictors
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_s = ss.transform(X_train)
    X_test_s = ss.transform(X_test)
    
    # fit
    model.fit(X_train_s, y_train)
    
    # Evaluate: predict
    
    y_pred = model.predict(X_test_s)
    y_true = y_test
    
    mean_square_error = np.sqrt(sklearn.metrics.mean_squared_error(y_true, y_pred))
    
    # Evaluate: score
    score = model.score(X_test_s, y_test)
    
    return {'Score (R^2)': score.mean(), 'MSE': mean_square_error}

### Great Brittan / United Kingdom

**Lasso #1 - TF-IDF + all coefs**

In [167]:
# Declare indep and dep
X_train_gb = gb_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_gb = gb_dep_train
X_test_gb = gb_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_gb = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_gb, y_train_gb)

{'best_params': {'alpha': 0.014174741629268055, 'selection': 'cyclic'},
 'best_score': 0.15992337416126728}

In [168]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0142, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_gb, X_test_gb, y_train_gb, y_test_gb)

{'MSE': 0.20965856308450798, 'Score (R^2)': 0.19824536945416926}

In [169]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_gb.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy    0.070752
nigga     0.013034
yes       0.011485
niggas    0.009455
lies      0.009167
girl      0.008916
little    0.008107
sexy      0.007695
til       0.007196
Mode      0.006588
wanted    0.006557
piece     0.006490
year      0.006062
hot       0.005779
scared    0.005671
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [170]:
# Declare indep and dep
X_train_gb2 = gb_indep_train_tvec[['Energy', 'nigga', 'yes', 'niggas', 'lies', 'girl', 'little', 'sexy',
                                'til', 'Mode']]
y_train_gb2 = gb_dep_train
X_test_gb2 = gb_indep_test_tvec[['Energy', 'nigga', 'yes', 'niggas', 'lies', 'girl', 'little', 'sexy',
                                'til', 'Mode']]
y_test_gb2 = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_gb2, y_train_gb2)

{'best_params': {'alpha': 0.003511191734215131, 'selection': 'random'},
 'best_score': 0.20992415971407796}

In [171]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0035, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_gb2, X_test_gb2, y_train_gb2, y_test_gb2)

{'MSE': 0.21431902256172267, 'Score (R^2)': 0.16220511069342944}

**Lasso #3 - TF-IDF + top 3 coefs**

In [172]:
# Declare indep and dep
X_train_gb3 = gb_indep_train_tvec[['Energy', 'nigga', 'yes']]
y_train_gb3 = gb_dep_train
X_test_gb3 = gb_indep_test_tvec[['Energy', 'nigga', 'yes']]
y_test_gb3 = gb_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_gb3, y_train_gb3)

{'best_params': {'alpha': 0.003511191734215131, 'selection': 'random'},
 'best_score': 0.1586682563987102}

In [173]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0035, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_gb3, X_test_gb3, y_train_gb3, y_test_gb3)

{'MSE': 0.21739922648868718, 'Score (R^2)': 0.1379503974694909}

COMMENT: Best Lasso model for GB is nr 1

### Italy

**Lasso #1 - TF-IDF + all coefs**

In [174]:
# Declare indep and dep
X_train_it = it_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_it = it_dep_train
X_test_it = it_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_it = it_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_it, y_train_it)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'cyclic'},
 'best_score': 0.09839733718258148}

In [175]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0163, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_it, X_test_it, y_train_it, y_test_it)

{'MSE': 0.1963286403446403, 'Score (R^2)': 0.14182794084676598}

In [176]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_it.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy     0.055309
oh         0.008944
baby       0.007454
yes        0.006948
davvero    0.006784
slow       0.006589
ogni       0.006167
girl       0.005481
pa         0.004603
hide       0.003974
stesso     0.003801
colpa      0.003487
quello     0.003412
domani     0.003121
vuoto      0.003061
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [177]:
# Declare indep and dep
X_train_it2 = it_indep_train_tvec[['Energy', 'oh', 'baby', 'yes', 'davvero', 'slow', 'ogni', 'girl', 'pa', 'hide']]
y_train_it2 = it_dep_train
X_test_it2 = it_indep_test_tvec[['Energy', 'oh', 'baby', 'yes', 'davvero', 'slow', 'ogni', 'girl', 'pa', 'hide']]
y_test_it2 = it_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_it2, y_train_it2)

{'best_params': {'alpha': 0.001, 'selection': 'cyclic'},
 'best_score': 0.18054457208343117}

In [178]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg2, X_train_it2, X_test_it2, y_train_it2, y_test_it2)

{'MSE': 0.19911841636100364, 'Score (R^2)': 0.11726588376323954}

**Lasso #3 - TF-IDF  + top 3 coefs**

In [179]:
# Declare indep and dep
X_train_it3 = it_indep_train_tvec[['Energy', 'oh', 'baby']]
y_train_it3 = it_dep_train
X_test_it3 = it_indep_test_tvec[['Energy', 'oh', 'baby']]
y_test_it3 = it_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_it3, y_train_it3)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.11933989374598247}

In [180]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_it3, X_test_it3, y_train_it3, y_test_it3)

{'MSE': 0.2000009103606321, 'Score (R^2)': 0.10942397884882317}

COMMENT: Best Lasso model for IT is nr 1

### US

**Lasso #1 - TF-IDF + all coefs**

In [181]:
# Declare indep and dep
X_train_us = us_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_us = us_dep_train
X_test_us = us_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_us = us_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000)

get_best_hype(lassoreg, grid, X_train_us, y_train_us)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'cyclic'},
 'best_score': 0.16685797058117727}

In [182]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000, alpha=0.0163, selection='cyclic')

# call function
evaluate_model(lassoreg, X_train_us, X_test_us, y_train_us, y_test_us)

{'MSE': 0.2039263436348886, 'Score (R^2)': 0.18856729273094788}

In [183]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_us.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy     0.071159
baby       0.017566
bitch      0.016582
rollie     0.009700
cake       0.009664
hey        0.006862
snow       0.006845
ground     0.006151
fucking    0.004809
askin      0.004478
bells      0.003999
sure       0.002857
mom        0.002627
shit       0.002469
soul       0.002452
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [184]:
# Declare indep and dep
X_train_us2 = us_indep_train_tvec[['Energy', 'baby', 'bitch', 'rollie', 'cake', 'hey', 'snow', 'ground',
                                'fucking', 'askin']]
y_train_us2 = us_dep_train
X_test_us2 = us_indep_test_tvec[['Energy', 'baby', 'bitch', 'rollie', 'cake', 'hey', 'snow', 'ground',
                                'fucking', 'askin']]
y_test_us2 = us_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_us2, y_train_us2)

{'best_params': {'alpha': 0.004037017258596553, 'selection': 'random'},
 'best_score': 0.24548921646298186}

In [185]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0040, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_us2, X_test_us2, y_train_us2, y_test_us2)

{'MSE': 0.21311377758339578, 'Score (R^2)': 0.11380580892919079}

**Lasso #3 - TF-IDF + top 3 coefs**

In [188]:
# Declare indep and dep
X_train_us3 = us_indep_train_tvec[['Energy', 'baby', 'bitch']]
y_train_us3 = us_dep_train
X_test_us3 = us_indep_test_tvec[['Energy', 'baby', 'bitch']]
y_test_us3 = us_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_us3, y_train_us3)

{'best_params': {'alpha': 0.001, 'selection': 'cyclic'},
 'best_score': 0.1915458239435362}

In [189]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg3, X_train_us3, X_test_us3, y_train_us3, y_test_us3)

{'MSE': 0.2062007590577543, 'Score (R^2)': 0.1703663406620386}

COMMENT: Best Lasso model for US is nr 1

### Deutchland / Germany

**Lasso #1 - TF-IDF + all coefs**

In [191]:
# Declare indep and dep
X_train_de = de_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_de = de_dep_train
X_test_de = de_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_de = de_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_de, y_train_de)

{'best_params': {'alpha': 0.014174741629268055, 'selection': 'random'},
 'best_score': 0.10260319639826802}

In [192]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0142, selection='random')

# call function
evaluate_model(lassoreg, X_train_de, X_test_de, y_train_de, y_test_de)

{'MSE': 0.19702451654502356, 'Score (R^2)': 0.13942174669991036}

In [193]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_de.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy    0.053144
snow      0.016295
seele     0.006984
hide      0.006081
bit       0.006004
baby      0.005770
stop      0.005340
mi        0.005261
tryin     0.004331
jahr      0.004280
deep      0.004218
schieß    0.004043
alles     0.004039
dicka     0.003946
ne        0.003731
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [194]:
# Declare indep and dep
X_train_de2 = de_indep_train_tvec[['Energy', 'snow', 'seele', 'hide', 'bit', 'baby', 'stop', 'mi',
                                'tryin', 'jahr']]
y_train_de2 = de_dep_train
X_test_de2 = de_indep_test_tvec[['Energy', 'snow', 'seele', 'hide', 'bit', 'baby', 'stop', 'mi',
                                'tryin', 'jahr']]
y_test_de2 = de_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_de2, y_train_de2)

{'best_params': {'alpha': 0.001, 'selection': 'cyclic'},
 'best_score': 0.17605461654719518}

In [195]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg2, X_train_de2, X_test_de2, y_train_de2, y_test_de2)

{'MSE': 0.2059957524275389, 'Score (R^2)': 0.05926704713528552}

**Lasso #3 - TF-IDF + top 3 coefs**

In [197]:
# Declare indep and dep
X_train_de3 = de_indep_train_tvec[['Energy', 'snow', 'seele']]
y_train_de3 = de_dep_train
X_test_de3 = de_indep_test_tvec[['Energy', 'snow', 'seele']]
y_test_de3 = de_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_de3, y_train_de3)

{'best_params': {'alpha': 0.001, 'selection': 'cyclic'},
 'best_score': 0.12593917028658846}

In [198]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg3, X_train_de3, X_test_de3, y_train_de3, y_test_de3)

{'MSE': 0.20209583173775286, 'Score (R^2)': 0.09454986315531066}

COMMENT: Best Lasso model for DE is nr 1

### Canada

**Lasso #1 - TF-IDF + all coefs**

In [199]:
# Declare indep and dep
X_train_ca = ca_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_ca = ca_dep_train
X_test_ca = ca_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_ca = ca_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_ca, y_train_ca)

{'best_params': {'alpha': 0.01873817422860384, 'selection': 'random'},
 'best_score': 0.13539154870271772}

In [200]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0187, selection='random')

# call function
evaluate_model(lassoreg, X_train_ca, X_test_ca, y_train_ca, y_test_ca)

{'MSE': 0.2234710335512767, 'Score (R^2)': 0.09803228704669986}

In [201]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_ca.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy              0.061516
sing                0.016041
baby                0.012496
rollie              0.007170
Acousticness        0.006309
year                0.005700
fucked              0.005159
Instrumentalness    0.004609
ye                  0.003095
brand               0.003008
picture             0.002939
mistletoe           0.002397
cake                0.002248
real                0.001954
nick                0.001210
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [203]:
# Declare indep and dep
X_train_ca2 = ca_indep_train_tvec[['Energy', 'sing', 'baby', 'rollie', 'Acousticness', 'year', 'fucked', 
                                   'Instrumentalness', 'ye', 'brand']]
y_train_ca2 = ca_dep_train
X_test_ca2 = ca_indep_test_tvec[['Energy', 'sing', 'baby', 'rollie', 'Acousticness', 'year', 'fucked', 
                                   'Instrumentalness', 'ye', 'brand']]
y_test_ca2 = ca_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_ca2, y_train_ca2)

{'best_params': {'alpha': 0.0015199110829529332, 'selection': 'random'},
 'best_score': 0.24688227138352511}

In [204]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0015, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_ca2, X_test_ca2, y_train_ca2, y_test_ca2)

{'MSE': 0.23511120386212514, 'Score (R^2)': 0.001621621693280506}

**Lasso #3 - TF-IDF + top 3 coefs**

In [205]:
# Declare indep and dep
X_train_ca3 = ca_indep_train_tvec[['Energy', 'sing', 'baby']]
y_train_ca3 = ca_dep_train
X_test_ca3 = ca_indep_test_tvec[['Energy', 'sing', 'baby']]
y_test_ca3 = ca_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_ca3, y_train_ca3)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.16892214749704096}

In [206]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_ca3, X_test_ca3, y_train_ca3, y_test_ca3)

{'MSE': 0.23812187229738607, 'Score (R^2)': -0.024111148255464124}

COMMENT: Best Lasso model for CA is nr 1

### Netherlands

**Lasso #1 - TF-IDF + all coefs**

In [207]:
# Declare indep and dep
X_train_nl = nl_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_nl = nl_dep_train
X_test_nl = nl_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_nl = nl_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000)

get_best_hype(lassoreg, grid, X_train_nl, y_train_nl)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'random'},
 'best_score': 0.17853784719484486}

In [208]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000, alpha=0.0163, selection='random')

# call function
evaluate_model(lassoreg, X_train_nl, X_test_nl, y_train_nl, y_test_nl)

{'MSE': 0.20337672125782755, 'Score (R^2)': 0.10295774896764186}

In [209]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_nl.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy     0.078235
bells      0.012143
Mode       0.010153
baby       0.008322
weg        0.006981
let        0.006770
town       0.006758
apart      0.005094
saaf       0.005025
kan        0.004808
moest      0.004233
goes       0.004068
forever    0.004048
praten     0.003035
drugs      0.002930
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [210]:
# Declare indep and dep
X_train_nl2 = nl_indep_train_tvec[['Energy', 'bells', 'Mode', 'baby', 'weg', 'let', 'town', 'apart',
                                'saaf', 'kan']]
y_train_nl2 = nl_dep_train
X_test_nl2 = nl_indep_test_tvec[['Energy', 'bells', 'Mode', 'baby', 'weg', 'let', 'town', 'apart',
                                'saaf', 'kan']]
y_test_nl2 = nl_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_nl2, y_train_nl2)

{'best_params': {'alpha': 0.002009233002565048, 'selection': 'random'},
 'best_score': 0.264407184215115}

In [211]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0020, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_nl2, X_test_nl2, y_train_nl2, y_test_nl2)

{'MSE': 0.21438045293241847, 'Score (R^2)': 0.003262531682686198}

**Lasso #3 - TF-IDF + top 3 coefs**

In [212]:
# Declare indep and dep
X_train_nl3 = nl_indep_train_tvec[['Energy', 'bells', 'Mode']]
y_train_nl3 = nl_dep_train
X_test_nl3 = nl_indep_test_tvec[['Energy', 'bells', 'Mode']]
y_test_nl3 = nl_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_nl3, y_train_nl3)

{'best_params': {'alpha': 0.0023101297000831605, 'selection': 'random'},
 'best_score': 0.2123807188625051}

In [213]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0023, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_nl3, X_test_nl3, y_train_nl3, y_test_nl3)

{'MSE': 0.20593984235300059, 'Score (R^2)': 0.08020473883587276}

COMMENT: Best Lasso model for NL is nr 1

### Australia

**Lasso #1 - TF-IDF + all coefs**

In [214]:
# Declare indep and dep
X_train_au = au_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_au = au_dep_train
X_test_au = au_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_au = au_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000)

get_best_hype(lassoreg, grid, X_train_au, y_train_au)

{'best_params': {'alpha': 0.021544346900318846, 'selection': 'random'},
 'best_score': 0.14393660997144453}

In [215]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=10000, alpha=0.0215, selection='random')

# call function
evaluate_model(lassoreg, X_train_au, X_test_au, y_train_au, y_test_au)

{'MSE': 0.2051367498577839, 'Score (R^2)': 0.17618254898908525}

In [216]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_au.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy       0.065373
sing         0.010464
alright      0.009373
baby         0.007071
ring         0.006150
apart        0.005182
attention    0.005085
ride         0.002789
grab         0.002744
perfect      0.002039
mess         0.001921
hate         0.001779
road         0.001352
forget       0.001234
chance       0.001023
dtype: float64

**Lasso #2 - TF-IDF + top 10 coefs**

In [217]:
# Declare indep and dep
X_train_au2 = au_indep_train_tvec[['Energy', 'sing', 'alright', 'baby', 'ring', 'apart', 'attention', 'ride',
                                'grab', 'perfect']]
y_train_au2 = au_dep_train
X_test_au2 = au_indep_test_tvec[['Energy', 'sing', 'alright', 'baby', 'ring', 'apart', 'attention', 'ride',
                                'grab', 'perfect']]
y_test_au2 = au_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_au2, y_train_au2)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.23711511623379708}

In [218]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg2, X_train_au2, X_test_au2, y_train_au2, y_test_au2)

{'MSE': 0.22352020510318546, 'Score (R^2)': 0.021912696811022148}

**Lasso #3 - TF-IDF + top 3 coefs**

In [219]:
# Declare indep and dep
X_train_au3 = au_indep_train_tvec[['Energy', 'sing', 'alright']]
y_train_au3 = au_dep_train
X_test_au3 = au_indep_test_tvec[['Energy', 'sing', 'alright']]
y_test_au3 = au_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_au3, y_train_au3)

{'best_params': {'alpha': 0.0013219411484660286, 'selection': 'random'},
 'best_score': 0.1792556638259798}

In [220]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0013, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_au3, X_test_au3, y_train_au3, y_test_au3)

{'MSE': 0.221388924490036, 'Score (R^2)': 0.040476031666445644}

COMMENT: Best Lasso model for AU is nr 1

### France

**Lasso #1 - TF-IDF + all coefs**

In [221]:
# Declare indep and dep
X_train_fr = fr_indep_train_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_train_fr = fr_dep_train
X_test_fr = fr_indep_test_tvec.drop(['Track Name', 'Artist', 'ID', 'Lyrics', 'Valence'], axis=1)
y_test_fr = fr_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg, grid, X_train_fr, y_train_fr)

{'best_params': {'alpha': 0.016297508346206444, 'selection': 'random'},
 'best_score': 0.15445741364318494}

In [222]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0163, selection='random')

# call function
evaluate_model(lassoreg, X_train_fr, X_test_fr, y_train_fr, y_test_fr)

{'MSE': 0.1942837162484487, 'Score (R^2)': 0.19342004396947587}

In [223]:
# Look at the feature importance with coef_
pd.Series(dict(zip(X_train_fr.columns,lassoreg.coef_))).abs().sort_values(ascending=False).head(15)

Energy          0.074359
x2              0.016805
snow            0.009685
baby            0.006743
quartier        0.006072
Acousticness    0.005724
putain          0.003758
bout            0.003390
chance          0.002723
fine            0.002249
uh              0.001386
rêve            0.001292
jean            0.001150
arrive          0.000888
vise            0.000811
dtype: float64

In [ ]:
# Do not use x2, that is from the lyrics text and only showing if something is repeating

**Lasso #2 - CountVec + top 10 coefs**

In [224]:
# Declare indep and dep
X_train_fr2 = fr_indep_train_tvec[['Energy', 'snow', 'baby', 'quartier', 'Acousticness', 'putain', 'bout', 'chance',
                                'fine', 'uh']]
y_train_fr2 = fr_dep_train
X_test_fr2 = fr_indep_test_tvec[['Energy', 'snow', 'baby', 'quartier', 'Acousticness', 'putain', 'bout', 'chance',
                                'fine', 'uh']]
y_test_fr2 = fr_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg2, grid, X_train_fr2, y_train_fr2)

{'best_params': {'alpha': 0.0015199110829529332, 'selection': 'cyclic'},
 'best_score': 0.21844452593278943}

In [225]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg2 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.0015, 
                  selection='cyclic')

# call function
evaluate_model(lassoreg2, X_train_fr2, X_test_fr2, y_train_fr2, y_test_fr2)

{'MSE': 0.1949680203703617, 'Score (R^2)': 0.1877281823095921}

**Lasso #3 - TF-IDF + top 3 coefs**

In [226]:
# Declare indep and dep
X_train_fr3 = fr_indep_train_tvec[['Energy', 'snow', 'baby']]
y_train_fr3 = fr_dep_train
X_test_fr3 = fr_indep_test_tvec[['Energy', 'snow', 'baby']]
y_test_fr3 = fr_dep_test

# GridSearchCV
grid = {'alpha': np.logspace(-3, 3, 100),
        'selection' : ('cyclic', 'random')}

lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000)

get_best_hype(lassoreg3, grid, X_train_fr3, y_train_fr3)

{'best_params': {'alpha': 0.001, 'selection': 'random'},
 'best_score': 0.1682811115321798}

In [227]:
# chose model and use best hyperparameters (from gridsearchCV)
lassoreg3 = Lasso(random_state=24, fit_intercept=True, normalize=False, max_iter=1000, alpha=0.001, 
                  selection='random')

# call function
evaluate_model(lassoreg3, X_train_fr3, X_test_fr3, y_train_fr3, y_test_fr3)

{'MSE': 0.19730490998031316, 'Score (R^2)': 0.16813968331479623}

COMMENT: Best Lasso model for FR is nr 1

**Summary TF-IDF - best model/score per country**<BR />
gb - R2 = 0.19824536945416926 (all coefs)<BR />
it - R2 = 0.14182794084676598 (all coefs)<BR />
us - R2 = 0.18856729273094788 (all coefs)<BR />
de - R2 = 0.13942174669991036 (all coefs)<BR />
ca - R2 = 0.09803228704669986 (all coefs)<BR />
nl - R2 = 0.10295774896764186 (all coefs)<BR />
au - R2 = 0.17618254898908525 (all coefs)<BR />
fr - R2 = 0.19342004396947587 (all coefs)

*The score and how many varables to choose (and which ones) varies from country to country*

**Summary over CountVec and TF-IDF**<BR />
The best model in total for every country:<BR />
gb - TF-IDF: R2 = 0.19824536945416926 (all coefs)<BR />
it - CountVec: R2 = 0.14182794084676598 (all coefs)<BR />
us - TF-IDF: R2 = 0.18856729273094788 (all coefs)<BR />
de - CountVec: R2 = 0.1676239483701173 (top 10 coefs)<BR />
ca - TF-IDF: R2 = 0.09803228704669986 (all coefs)<BR />
nl - TF-IDF: R2 = 0.10295774896764186 (all coefs)<BR />
au - CountVec: R2 = 0.19230354139175532 (top 3 coefs)<BR />
fr - TF-IDF: R2 = 0.19342004396947587 (all coefs)

*The score and how many varables to choose (and which ones) varies from country to country. Also the choice btw TF-IDF och CountVec seem to vary.*